# 테스트 데이터 생성하기


아래 코드의 bucketName에 SAM 배포시 생성 된 forecast bucket 이름을 넣어 줍니다. 

예시> `bucketName='forecast-demo-forecastbucket-1dcxlj3j2zvc3'`

In [1]:
import pandas as pd 
import numpy as np
import random
import datetime
import boto3
from tqdm import tqdm

bucketName= #forecast bucket 이름을 넣어주세요.

2018년부터 2019년까지 2년간의 테스트 데이터를 생성 합니다.
일주일 중 하루 또는 이틀(주말) 판매량이 증가하는 상품 데이터(각 500개 아이템)와 꾸준히 판매량이 증가하는 상품 데이터(1000개 아이템), 꾸준한 판매량을 유지하는 상품 데이터를 임의로 만들어 봅니다.


In [ ]:
region='ap-northeast-2'
session = boto3.Session(region_name=region) 
s3 = session.client('s3')
notebook_rawdata_path='data/raw_data/forecast_data.csv'
forecast_rawdata_path='raw_data/forecast_data.csv'

df_total=pd.DataFrame()
# dates = pd.date_range('20180101', periods=365*2)
start = datetime.datetime.strptime("2018/01/01", "%Y/%m/%d")
end = datetime.datetime.strptime("2019/12/31", "%Y/%m/%d")
date_generated = [start + datetime.timedelta(days=x) for x in range(0, (end-start).days+1)]

def random_date(first_date, second_date):
    first_timestamp = int(first_date.timestamp())
    second_timestamp = int(second_date.timestamp())
    random_timestamp = random.randint(first_timestamp, second_timestamp)
    return datetime.fromtimestamp(random_timestamp)


n=1 #Base number
item_num1=500 #Numbers of items for 1day promotion
item_num2=500 #Numbers of items for 2day promotion
item_num3=1000 #Numbers of itmes for steady increasing trend
item_num4=500 #Numbers of items for stable trend

# Generate special day(1day promotion a week) trend data 
for i in tqdm(range(item_num1)):
    data_list = []
    
    tg_date = random.randrange(6)

    tg_date_value = random.randrange(200, 500)
    tg_date_value_gap = random.randrange(10, 200)
    
    nm_date_value = random.randrange(150)
    nm_date_value_gap = random.randrange(1,50)
    
    for day in date_generated:
        if(day.weekday() == tg_date):
            data_list.append(random.sample(range(tg_date_value,tg_date_value+tg_date_value_gap),1)[0])
        else:
            data_list.append(random.sample(range(nm_date_value,nm_date_value+nm_date_value_gap),1)[0])
    
    df_1day = pd.DataFrame({'timestamp': [date.strftime('%Y/%m/%d') for date in date_generated],
                            'item_id': 'item'+str(n+i),
                            'target_value': data_list})
    
    df_total=df_total.append(df_1day)


# Generate special day(2day promotion a week) trend data 
n=n+item_num1
for i in tqdm(range(item_num2)):
    data_list = []
    
    list = [0,1,2,3,4,5,6]
    tg_date_list = random.sample(list,2)

    tg_date_value = random.randrange(200, 500)
    tg_date_value_gap = random.randrange(10, 200)
    
    nm_date_value = random.randrange(150)
    nm_date_value_gap = random.randrange(1,50)

    for day in date_generated:
        if(day.weekday() == tg_date_list[0] or day.weekday() == tg_date_list[1]):
            data_list.append(random.sample(range(tg_date_value,tg_date_value+tg_date_value_gap),1)[0])
        else:
            data_list.append(random.sample(range(nm_date_value,nm_date_value+nm_date_value_gap),1)[0])

    df_2day = pd.DataFrame({'timestamp': [date.strftime('%Y/%m/%d') for date in date_generated],
                            'item_id': 'item'+str(n+i),
                            'target_value': data_list})
    
    df_total=df_total.append(df_2day)
    
# Generate steady increasing trend data
n=n+item_num2
for i in tqdm(range(item_num3)):
    results = np.arange(365*2)*random.uniform(0, 0.5)
    noise = np.random.randn(365*2)*random.randrange(50)
    noise = [abs(i) for i in noise]

    results += noise
    results = [int(i) for i in results]
    
    df_incline = pd.DataFrame({'timestamp': [date.strftime('%Y/%m/%d') for date in date_generated],
                               'item_id': 'item'+str(n+i),
                               'target_value': results})
    
    df_total=df_total.append(df_incline)

# Generate stable trend data
n=n+item_num3
for i in tqdm(range(item_num4)):
    r_range = random.randrange(150, 1000)
    df_normal = pd.DataFrame({'timestamp': [date.strftime('%Y/%m/%d') for date in date_generated],
                              'item_id': 'item'+str(n+i),
                              'target_value': [random.randrange(r_range-100,r_range) for j in range(365*2)]})
    df_total=df_total.append(df_normal)


생성된 데이터를 csv 파일 형태로 forecast bucket의 raw_data 폴더에 업로드 합니다.

In [ ]:
df_total.to_csv(notebook_rawdata_path, index=False, header=True)
s3.upload_file(notebook_rawdata_path, bucketName, forecast_rawdata_path)
print('The csv file is uploaded.')